<a href="https://colab.research.google.com/github/OSegun/Deep-Learning-Projects/blob/main/Iris_Dataset_with_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from sklearn.datasets import load_iris
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns

In [31]:
iris = load_iris()

X = iris.data
y = iris.target

In [32]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [46]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=22)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

In [68]:
class IrisDataset(Dataset):

  def __init__(self, X_train, y_train):
    super().__init__()
    self.X_train = torch.from_numpy(X_train)
    self.y_train = torch.from_numpy(y_train).type(torch.LongTensor)

  def __len__(self):
    return len(self.X_train)

  def __getitem__(self, index):
    return self.X_train[index], self.y_train[index]


iris_df = IrisDataset(X_train=X_train, y_train=y_train)
train_dataloader = DataLoader(dataset=iris_df, batch_size=32)

In [53]:
class ModelClassifer(nn.Module):

  def __init__(self, features, hidden, classes):
    super().__init__()
    self.layer1 = nn.Linear(features, hidden)
    self.layer2 = nn.Linear(hidden, classes)
    self.output = nn.LogSoftmax(dim=1)


  def forward(self, x):
    x = self.layer1(x)
    x = torch.sigmoid(x)
    x = self.layer2(x)
    x = self.output(x)
    return x

features = iris_df.X_train.shape[1]
hidden = 12
classes = len(iris_df.y_train.unique())

In [72]:
def train_model(model, criterion, epochs, optimzer):
  losses = []
  for epoch in range(epochs):
    for x,y in train_dataloader:
      optimzer.zero_grad()
      pred = model(x)
      loss = criterion(pred, y)
      loss.backward()
      optimzer.step()

  losses.append(float(loss.data.detach().numpy()))

  return f"Loss: {losses}"

In [73]:
model = ModelClassifer(
    features=features,
    hidden=hidden,
    classes=classes
    )

criterion = nn.CrossEntropyLoss()
optimzer = torch.optim.SGD(model.parameters(), lr=0.001)

train_model(
    model=model,
    criterion=criterion,
    epochs=10000,
    optimzer=optimzer
    )

'Loss: [0.32268592715263367]'

In [74]:
#sns.lineplot(x=range(10000), y=losses)

In [75]:
X_test = torch.from_numpy(X_test)

with torch.no_grad():
  y_pred = model(X_test)
  y_pred = torch.max(y_pred.data, 1)

In [80]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

accuracy_score(y_test, y_pred.indices)

1.0

In [79]:
confusion_matrix(y_test, y_pred.indices)

array([[ 6,  0,  0],
       [ 0, 10,  0],
       [ 0,  0, 14]])

In [82]:
print(classification_report(y_test, y_pred.indices))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        14

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

